Stamping files

In [ ]:
from pathlib import Path
import json
import os
import sys
from pathlib import Path

# Add the parent directory of 'multistamper/' to sys.path
sys.path.append(str(Path().resolve()))
from multistamper.core import LocalDataSource, Pipeline, ApiClient
from multistamper.stamp_cli import run_pipeline, setup, main
from dotenv import load_dotenv

load_dotenv()

In [ ]:
users_folder_path = "users"
users_folder = Path(users_folder_path).resolve()
assert users_folder.exists(), f"Users folder not found: {USERS_FOLDER}"

In [ ]:
api_url = os.getenv("VBASE_API_URL")

In [ ]:
assert api_url, "Missing VBASE_API_URL in .env"

Run pipeline and stamp files

Preview files, Do you want to stamp these files?

In [ ]:
run_pipeline(api_url, users_folder, False)

Stamp files

In [ ]:
run_pipeline(api_url, users_folder, True)